In [20]:
import torch
import transformers

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

import sys
import os

# 현재 스크립트의 위치 기준으로 상위 디렉토리 추가
sys.path.append(os.path.abspath("colossalai_ChatGPT_230319"))

import torch
import torch.nn.functional as F

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

from abc import ABC
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch
import torch.nn as nn
from chatgpt.trainer.strategies import NaiveStrategy, Strategy
from torch.optim import Optimizer
from chatgpt.trainer.utils import is_rank_0

from transformers import GPT2LMHeadModel

Torch version:1.12.1
Cuda version: 11.3
transformers version: 4.28.0
GPU 사용 가능여부: True


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
input_txt = "3+5 = ?"

tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
max_length = 128

output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

3+5 =?? (... ).
그런데 이 녀석이 뭔지 알 수 없는 건지...
아무튼 이녀석의 정체는 닌자 거북이.</d> 2017년 12월 11일부터 2018년 12월 31일까지 방영된 SBS 월화드라마.
2017년 11월 12일부터 2020년 1월 15일까지 매주 금요일 오후 7시 40분에 방송된다.</d> ##당 내부 선거가 아닌 대한민국 선출직 공직자 선거 결과만 기록한다.
선거종류 득표수 ( 득표율 ) 비고 1996 민주통합당 낙선 ( 1위 )
제19대 국회의원 선거 ( 경기도 수원시


In [4]:
import json 
data_path_1_SFT = './data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

In [6]:
data_path_2_RM = './data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

# SFT

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [6]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [7]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [8]:
train_dataset = SFT_dataset(data_path_1_SFT='./data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [9]:
# input은 그대로
print('input : %s' % tokenizer.decode(train_dataset.input_ids[0]))

# output은 ignore_index 제거 후 디코딩
label_ids = [id for id in train_dataset.labels[0] if id != -100]
print('output: %s' % tokenizer.decode(label_ids))

input : ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>
output: '저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>


In [13]:
training_args = TrainingArguments(
    output_dir="./test",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [14]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  501217 KB |  701897 KB |   16713 MB |   16224 MB |\n|       from large pool |  488448 KB |  689082 KB |   13740 MB |   13263 MB |\n|       from small pool |   12769 KB |   84866 KB |    2972 MB |    2960 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |  501217 KB |  701897 KB |   16713 MB |   16224 MB |\n|       from large pool |  488448 KB |  689082 KB |

In [15]:
model.gradient_checkpointing_enable()

In [16]:
trainer.train()
model.save_pretrained('./output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,2.959200
1000,2.810200
1500,2.771300
2000,2.245900
2500,2.277600
3000,2.287300
3500,1.819900
4000,1.892100
4500,1.900500
5000,1.508700


In [17]:
generator = pipeline('text-generation', model='./output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 실제 매장에 대한 정보는 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 따라서 한우의 경우 한우, 쇠고기의 경우 한우를 사용하는 경우가 많습니다. 참고: 참고: "불고기용

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'1953년입니다. 리처드 닉슨은 47대 부통령직을 수행했습니다.\n\n1953년 대선 캠프 도중 대통령 후보인 드와이트 D. 아이젠하워와 부통령 후보로 출마할 때 닉슨은 부통령 후보로 출마한 리처드 니콜슨을 지지하며 대선 캠프에서 중요한 역할을

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다.: DOMY (Department of Okay): Michael Olivery James:\n\nCould you please translation

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과 실외 활동 자제를 권장합니다. 외출 후에는 반드시 마스크를 착용하시는 것이 좋습니다. 참고용으로만 활용하시면 더 나은 미세먼지 농도를 확인할 수 있습니다. 참고용으로는 미세먼지 측정소에서


# RM

In [9]:
torch.cuda.empty_cache()

In [10]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [11]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [12]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
with open('./data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [14]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [15]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 869.88it/s]


In [72]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [26]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=2,
                             max_epochs=10)

In [27]:
trainer.fit(use_lora=0)

model.save_pretrained('./output_2_RM')

Train step of epoch 0:   1%|          | 4/500 [00:02<04:28,  1.85it/s, loss=0.411]
... (tqdm 출력 생략)

In [28]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.2


In [29]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: 0.3


# PPO

In [16]:
torch.cuda.empty_cache()

In [17]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [18]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='./output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='./output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [33]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [34]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [35]:
with open('./data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [36]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [37]:
len(list_prompt)

12000

In [38]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=10,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [39]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

Train epoch [2/10]:  33%|███▎      | 1/3 [00:00<00:01,  1.71it/s, actor_loss=0, critic_loss=0.0527]
... (tqdm 출력 생략)

In [40]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이기 때문에 실제로 매장을 방문하지 않습니다. 그러나 일반적으로 불고기용 고기는 한우를 사용하는 가정에서 구매하기 때문에 한우의사료 사용량이 많기 때문에 한우도매장에서 구매할 가능성이 높습니다. 감사를 표시하는 광고나 포스터를 활용하여, 한우를 사랑하는 국민들의 의견을 수렴하고 싶습니다. 감사함을 나타내는 문의는 사회적 이슈에 적극적으로 대처하는 것이 중요하다고 생각됩니다. 감사합니다! 자리는 다양한 분야의 정보와 교류가 진행되는 가운데에서도, 우리 사회에 긍정적인 영향을 미치는 중요한 역할을 합니다. 감사합니다. 국민의 사랑과 지지를 받으며, 한국 사회에서는 중요한 역할을 하도록 하겠습니다. 의석을수록 좋은 글은 사회적 이슈에 적극적으로 대응하고, 더 많은 사람들에게 더 좋은 의견을 제시하기 위해 노력할 것입니다. 평의 가치와 더불어, 더 많은 정보를 창출하기 위해서는, 경제, 정치, 문화 등 다양한 분야를 이해하고 존중하는 것이 매우 중요합니다. 감사함에 대해 전하고 싶습니다. 찬사를 기념하는 많은 의견과 활동은 새로운 인권과 일자리를 찾는 데 매우 중요하다고 생각합니다. 감사함은 세상을 더 잘 이해하고, 사회적 변화와 함께 더 많은 사람들을 더 잘 이해하게 할 수 있도록 하는 좋은 방법일 것입니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'1953년입니다. 저는 닉슨이 47대 부통령직을 수행한 년도를 알 수 없습니다. 분석하면, 1976년 대선에서 닉슨이 부통령직을 수행한 후 대통령 선거에서 지지표가 부족해 출마하지 못하게 됐습니다. 추정으로는, 2017년 대선 당시 닉슨은 46대 부통령직을 수행한 것으로 알려져 있습니다. 정확히는 "대선 결과의 불확실성"이라는 의미로 해석되기 때문입니다.ographical corre 참여하긴 어렵지만, "39대 부통령", 

# 추가 코드 수정 : DPO 사용해보기

DPO(Direct Preference Optimization)는 LLM의 미세조정(fine-tuning) 방법 중 하나로, 특히 사용자 선호(preference)를 반영한 응답 생성을 목표로 하는 강화 학습 대안 기법

SFT-RM-PPO 과정 : 

prompt-response 쌍으로 지도학습된 sft모델

\+ 여러 개의 응답 중 어느 쪽이 더 나은지를 학습하는 rm 모델

\+ rm이 높게 평가하는 출력을 생성하는 ppo 모델

SFT-DPO 과정 :

기존과 동일하게 base 모델 생성

\+ RM 없이 사람이 평가한 (better, worse) 쌍으로 LLM 학습

-> 구조가 간결해지고 효율이 향상된다.

In [22]:
torch.cuda.empty_cache()

In [30]:
with NaiveStrategy().model_init_context():
    from transformers import GPT2LMHeadModel

    actor = GPT2LMHeadModel.from_pretrained("./output_1_SFT")
    
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
actor.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [31]:
def get_logps(model, input_ids, attention_mask):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # [B, T, V]

    log_probs = F.log_softmax(logits, dim=-1)  # [B, T, V]
    log_probs_target = log_probs.gather(2, input_ids.unsqueeze(-1)).squeeze(-1)  # [B, T]

    # padding 제외한 평균 log-prob
    seq_mask = attention_mask.bool()
    log_probs_masked = log_probs_target * seq_mask
    avg_log_probs = log_probs_masked.sum(dim=1) / seq_mask.sum(dim=1)
    return avg_log_probs  # [B]

def dpo_loss(
    model,
    prompt_input_ids, prompt_attention_mask,
    chosen_input_ids, chosen_attention_mask,
    rejected_input_ids, rejected_attention_mask,
    beta: float = 0.1,
):
    device = next(model.parameters()).device
    chosen_input_ids = chosen_input_ids.to(device)
    chosen_attention_mask = chosen_attention_mask.to(device)
    rejected_input_ids = rejected_input_ids.to(device)
    rejected_attention_mask = rejected_attention_mask.to(device)

    chosen_logps = get_logps(model, chosen_input_ids, chosen_attention_mask)
    rejected_logps = get_logps(model, rejected_input_ids, rejected_attention_mask)

    # DPO loss
    loss = -torch.nn.functional.logsigmoid(beta * (chosen_logps - rejected_logps)).mean()
    return loss


In [32]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

print(device)

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)





def dpo_collate_fn(batch):
    def to_cuda(t):
        return t.cuda(non_blocking=True)
    
    chosen_input_ids = to_cuda(torch.stack([item[0][0] for item in batch]))
    chosen_attn_mask = to_cuda(torch.stack([item[1][0] for item in batch]))
    rejected_input_ids = to_cuda(torch.stack([item[2][0] for item in batch]))
    rejected_attn_mask = to_cuda(torch.stack([item[3][0] for item in batch]))

    return {
        "prompt_input_ids": chosen_input_ids,
        "prompt_attention_mask": chosen_attn_mask,
        "chosen_input_ids": chosen_input_ids,
        "chosen_attention_mask": chosen_attn_mask,
        "rejected_input_ids": rejected_input_ids,
        "rejected_attention_mask": rejected_attn_mask,
    }


class DPOTrainer(ABC):
    def __init__(
        self,
        model,
        strategy: Strategy,
        optim: Optimizer,
        train_dataset,
        eval_dataset,
        beta: float = 0.1,
        batch_size: int = 1,
        max_epochs: int = 2,
        gradient_accumulation_steps: int = 1,  # ✅ 추가
    ) -> None:
        super().__init__()
        self.strategy = strategy
        self.epochs = max_epochs
        self.gradient_accumulation_steps = gradient_accumulation_steps  # ✅ 저장
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=dpo_collate_fn)
        self.eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn=dpo_collate_fn)

        self.model = strategy.setup_model(model)
        if "DDP" in str(self.strategy):
            self.model = self.model.module
        self.optimizer = strategy.setup_optimizer(optim, self.model)
        self.beta = beta
        self.train_losses = []

    def fit(self):
        epoch_bar = tqdm(range(self.epochs), desc='Train epoch', disable=not is_rank_0())
        global_step = 0  # ✅ 전역 step 추적

        for epoch in range(self.epochs):
            step_bar = tqdm(self.train_dataloader, desc=f'Train step of epoch {epoch}', disable=not is_rank_0())

            self.model.train()
            for step, batch in enumerate(step_bar):
                prompt = batch["prompt_input_ids"]
                prompt_mask = batch["prompt_attention_mask"]
                chosen = batch["chosen_input_ids"]
                chosen_mask = batch["chosen_attention_mask"]
                rejected = batch["rejected_input_ids"]
                rejected_mask = batch["rejected_attention_mask"]

                loss = dpo_loss(
                    self.model,
                    prompt_input_ids=prompt,
                    prompt_attention_mask=prompt_mask,
                    chosen_input_ids=chosen,
                    chosen_attention_mask=chosen_mask,
                    rejected_input_ids=rejected,
                    rejected_attention_mask=rejected_mask,
                    beta=self.beta,
                )

                # ✅ loss scaling for gradient accumulation
                loss = loss / self.gradient_accumulation_steps

                self.strategy.backward(loss, self.model, self.optimizer)

                # ✅ optimizer step and zero_grad() only every accumulation step
                if (step + 1) % self.gradient_accumulation_steps == 0:
                    self.strategy.optimizer_step(self.optimizer)
                    self.optimizer.zero_grad()

                #self.train_losses.append(loss.detach().cpu().item())
                
                step_bar.set_postfix({'loss': loss.item()})
                global_step += 1

            epoch_bar.update()
            step_bar.close()

        epoch_bar.close()


# Example usage
trainer = DPOTrainer(
    model=actor,
    strategy=NaiveStrategy(),
    optim=Adam(actor.parameters(), lr=5e-5),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    batch_size=2,                     # 여전히 작게 유지 가능
    max_epochs=10,
    beta=0.1,
    gradient_accumulation_steps=8     # ✅ 누적하여 16배 가상 배치 효과
)
trainer.fit()

1000
200
cuda


  0%|          | 0/200 [00:00<?, ?it/s]
... (tqdm 출력 생략)

In [40]:
# DPO 학습 후
actor.save_pretrained("./output_dpo")      # GPT2LMHeadModel 저장
tokenizer.save_pretrained("./output_dpo")        # Tokenizer 저장

('./output_dpo/tokenizer_config.json',
 './output_dpo/special_tokens_map.json',
 './output_dpo/vocab.json',
 './output_dpo/merges.txt',
 './output_dpo/added_tokens.json',
 './output_dpo/tokenizer.json')

In [33]:
torch.cuda.empty_cache()

In [50]:
actor = GPT2LMHeadModel.from_pretrained("./output_dpo").cuda()
actor.eval()

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    'output_dpo',
    bos_token='<s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    model_max_length=512,
    padding_side="right"
)

In [51]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(actor.device)

    outputs = actor.generate(
        input_ids=input_ids,
        max_length=128,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"[입력]: {input_text}\n[출력]: {output_text}\n")
    return output_text


인퍼런스 결과.

질문과 출력이 완전히 일치하거나, 

출력에 의미 없는 반복이 포함되어 있음.

질문-응답 쌍 형식이 잘못 구성되어있음을 시사.

In [52]:
PROMPT_DICT = {
    "prompt_input": "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?'
]

#list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]
for prompt in list_prompt:
    generation(prompt)


[입력]: 불고기용 고기 한우에요?
[출력]: 불고기용 고기 한우에요? 한개는에 ric 개는 개는 개는 개는 개는 개는 개는 개는 개는 개는 개는 개는   개는                                                  개는                        개는           

[입력]: 리처드 닉슨이 43대 부통령직을 수행한 년도는?
[출력]: 리처드 닉슨이 43대 부통령직을 수행한 년도는?                                                                                                                     

[입력]: 시카고 오헤어 국제공항은 어디에 있어
[출력]: 시카고 오헤어 국제공항은 어디에 있어 있고 있지 않고, 않고 있습니다. 있습니다.습니다.                                                                                                                 

[입력]: 오늘 미세먼지 어때?
[출력]: 오늘 미세먼지 어때? hahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahaha



데이터를 확인해보니, 질문과 응답이 잘 분리되지 않고, eos가 \<|endoftext|>로 되어있는 모습을 확인할 수 있었습니다.

In [57]:
# chosen input 디코딩
print("▶ Chosen decoded:")
print(tokenizer.decode(chosen_input_ids.squeeze().tolist(), skip_special_tokens=True))

# rejected input 디코딩
print("\n▶ Rejected decoded:")
print(tokenizer.decode(rejected_input_ids.squeeze().tolist(), skip_special_tokens=True))

▶ Chosen decoded:
보증금 500 정도인 곳으로 보여 주실 수 있어요?죄송합니다. 저는 AI 어시스턴트이기 때문에, 지역이나 상황에 따라 다양한 보증금 요건을 가진 장소들을 알지 못합니다. 하지만, 인터넷 검색이나 부동산 에이전트와 상담하여 원하는 지역에서 보증금이 500 정도인 장소를 찾을 수 있습니다. 감사합니다.<|endoftext|>

▶ Rejected decoded:
보증금 500 정도인 곳으로 보여 주실 수 있어요?보증금 500 원 이하의 월세 아파트는 매우 드물기 때문에 찾기가 어렵습니다. 가능한 경우는 작은 동네에 있는 작은 아파트를 찾는 것이 좋습니다. 또한, 보증금이 없는 월세 아파트를 찾는 것도 좋은 방법입니다.<|endoftext|>


eos 토큰이 '<', '|', 'end', 'o', 'f', 'te', 'x', 't', '|', '>' 같이 토큰화되어, eos 설정을 다시 수행했습니다.

In [86]:
sample = train_dataset[1]
chosen_input_ids = sample[0][0]
rejected_input_ids = sample[2][0]

# 디코딩
chosen_text = tokenizer.decode(chosen_input_ids.tolist(), skip_special_tokens=False)
rejected_text = tokenizer.decode(rejected_input_ids.tolist(), skip_special_tokens=False)

print("▶ Chosen decoded:\n", chosen_text)
print("\n▶ Rejected decoded:\n", rejected_text)

tokens = tokenizer.convert_ids_to_tokens(chosen_input_ids)
print("▶ Chosen tokens:\n", tokens)

▶ Chosen decoded:
 흑고래의 무게는 어느 정도야흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.<|endoftext|></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

In [ ]:
with NaiveStrategy().model_init_context():
    # 1. Tokenizer 정의 (eos_token을 '<|endoftext|>'로 지정)
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2',
        bos_token='</s>',
        eos_token='<|endoftext|>',  # ✅ 진짜 eos
        unk_token='</s>',
        pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )

    # 2. 모델 로드
    actor = GPT2LMHeadModel.from_pretrained("./output_1_SFT")

    # 3. Special token이 vocab에 없으면 추가됨 → 모델 사이즈 조정 필요
    actor.resize_token_embeddings(len(tokenizer))

    # 4. 초기 가중치 백업
    initial_model = deepcopy(actor)

질문과 대답 사이에 eos를 넣어주는 코드.

In [26]:
from copy import deepcopy

def prepare_dpo_concat_format(data, tokenizer):
    eos = tokenizer.eos_token  # 예: '</s>'
    new_data = []
    for sample in data:
        prompt = sample["prompt"]
        chosen = sample["chosen"]
        rejected = sample["rejected"]

        new_sample = {
            "prompt": "",  # prompt 제거
            "chosen": prompt + eos + chosen,
            "rejected": prompt + eos + rejected
        }
        new_data.append(new_sample)
    return new_data

# 적용
train_data = prepare_dpo_concat_format(total_data_ranking2chosen[:1000], tokenizer)
eval_data = prepare_dpo_concat_format(total_data_ranking2chosen[1000:1200], tokenizer)

# 기존 RewardDataset 그대로 사용
train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

100%|██████████| 200/200 [00:00<00:00, 1103.45it/s]


첫 eos (질문과 대답을 분리하는 부분)에서부터 마스킹을 수행하여

응답 부분만 loss 계산에 반영하게 됩니다.

In [24]:
def get_response_mask(input_ids, split_token_id=51200):
    mask = torch.ones_like(input_ids)
    for i in range(input_ids.size(0)):
        sep_indices = (input_ids[i] == split_token_id).nonzero(as_tuple=True)[0]
        if len(sep_indices) > 0:
            # ✅ 첫 번째 <|endoftext|> 이후부터 응답이므로 그 이전은 0으로 마스킹
            mask[i, :sep_indices[0] + 1] = 0
    return mask

# ========== Log-Probability 계산 함수 ==========
def get_logps(model, input_ids, attention_mask):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    log_probs = F.log_softmax(logits, dim=-1)
    log_probs_target = log_probs.gather(2, input_ids.unsqueeze(-1)).squeeze(-1)

    seq_mask = get_response_mask(input_ids).to(input_ids.device)
    log_probs_masked = log_probs_target * seq_mask
    avg_log_probs = log_probs_masked.sum(dim=1) / seq_mask.sum(dim=1)
    return avg_log_probs

# ========== DPO Loss 계산 ==========
def dpo_loss(
    model,
    prompt_input_ids, prompt_attention_mask,
    chosen_input_ids, chosen_attention_mask,
    rejected_input_ids, rejected_attention_mask,
    beta: float = 0.1,
):
    device = next(model.parameters()).device
    chosen_input_ids = chosen_input_ids.to(device)
    chosen_attention_mask = chosen_attention_mask.to(device)
    rejected_input_ids = rejected_input_ids.to(device)
    rejected_attention_mask = rejected_attention_mask.to(device)

    chosen_logps = get_logps(model, chosen_input_ids, chosen_attention_mask)
    rejected_logps = get_logps(model, rejected_input_ids, rejected_attention_mask)

    loss = -torch.nn.functional.logsigmoid(beta * (chosen_logps - rejected_logps)).mean()
    return loss

# ========== Collate Function ==========
def dpo_collate_fn(batch):
    def to_cuda(t): return t.cuda(non_blocking=True)
    return {
        "prompt_input_ids": to_cuda(torch.stack([item[0][0] for item in batch])),
        "prompt_attention_mask": to_cuda(torch.stack([item[1][0] for item in batch])),
        "chosen_input_ids": to_cuda(torch.stack([item[0][0] for item in batch])),
        "chosen_attention_mask": to_cuda(torch.stack([item[1][0] for item in batch])),
        "rejected_input_ids": to_cuda(torch.stack([item[2][0] for item in batch])),
        "rejected_attention_mask": to_cuda(torch.stack([item[3][0] for item in batch]))
    }

# ========== DPO Trainer ==========
class DPOTrainer(ABC):
    def __init__(self, model, train_dataset, eval_dataset, optimizer, beta=0.1, batch_size=2, max_epochs=1, gradient_accumulation_steps=1):
        self.model = model.cuda()
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=dpo_collate_fn)
        self.eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn=dpo_collate_fn)
        self.optimizer = optimizer
        self.beta = beta
        self.epochs = max_epochs
        self.gradient_accumulation_steps = gradient_accumulation_steps

    def fit(self):
        self.model.train()
        for epoch in range(self.epochs):
            progress_bar = tqdm(self.train_dataloader, desc=f"Epoch {epoch}")
            for step, batch in enumerate(progress_bar):
                loss = dpo_loss(self.model, **batch, beta=self.beta)
                loss = loss / self.gradient_accumulation_steps
                loss.backward()
                if (step + 1) % self.gradient_accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                progress_bar.set_postfix({"loss": loss.item()})


# ========== Example Usage ==========

sample = train_dataset[1]
chosen_input_ids = sample[0][0]
rejected_input_ids = sample[2][0]

# 디코딩
chosen_text = tokenizer.decode(chosen_input_ids.tolist(), skip_special_tokens=False)
rejected_text = tokenizer.decode(rejected_input_ids.tolist(), skip_special_tokens=False)

print("▶ Chosen decoded:\n", chosen_text)
print("\n▶ Rejected decoded:\n", rejected_text)

tokens = tokenizer.convert_ids_to_tokens(chosen_input_ids)
print("▶ Chosen tokens:\n", tokens)

eos_id = tokenizer.convert_tokens_to_ids("<|endoftext|>")
print(eos_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


▶ Chosen decoded:
 흑고래의 무게는 어느 정도야<|endoftext|> 흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.<|endoftext|></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>

In [25]:
# 5. GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
actor.to(device)

# assume train_dataset / eval_dataset are already defined
optimizer = Adam(model.parameters(), lr=5e-5)

trainer = DPOTrainer(
    model=actor,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    optimizer=optimizer,
    batch_size=2,
    max_epochs=5,
    beta=0.1,
    gradient_accumulation_steps=8
)
trainer.fit()

Epoch 4: 100%|██████████| 500/500 [06:08<00:00,  1.36it/s, loss=0.098] 


In [29]:
# ========== Inference Function ==========
def generation(input_text, model, tokenizer):
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors='pt').cuda()

    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=128,  # or max_length - input_len
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ▶ 프롬프트 제거: 응답만 남기기
    if decoded_output.startswith(input_text):
        decoded_output = decoded_output[len(input_text):].strip()

    print(f"\n[PROMPT]: {input_text}\n[OUTPUT]: {decoded_output}")
    return decoded_output


# 출력 결과 분석

전체적으로 토큰의 길이가 매우 길고 마치 eos를 잘 만들지 못하는 모습입니다.

그 원인을 추론해 보았을 때, SFT에서 eos를 학습할 때 eos_token='<\\s>'였는데,

DPO에서는 eos_token='<|endoftext|>'로 학습이 되어

추론에서 모델이 '<\\s>'를 만나도 종료가 되지 않았다고 생각할 수 있습니다.

이런 문제를 제외했을 때, rm-sft 과정 없이도 질문에 잘 응답하는 추론을 확인할 수 있습니다.

In [30]:
PROMPT_DICT = {"prompt_input": "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"}
list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    '3+5=?'
]

for prompt in list_prompt:
    formatted = PROMPT_DICT["prompt_input"].format_map({"prompt": prompt})
    generation(formatted, actor, tokenizer)


[PROMPT]: ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):
[OUTPUT]: '저는 AI이기 때문에 매장에 따라 다르기 때문에 정확한 답변을 드리기 어렵습니다. 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 한우, 쇠고기, 돼지고기 등 고기 한우는 사용되고, 쇠고기는 유통기한이 지난 식품으로, 한우도 다른 종류의 고기와 함께 사용됩니다. 불고기용 고기는 일반적으로 2-3일 정도, 불고기용 고기는 4회 정도 사용될 수 있습니다.下, \n\n하지만 한우, 쇠고기, 돼지고기 등 고기를 사용할 수 있는 많은 종류의 고기 중에서도, 고기를 양념하거나 추가하기 전에 미리 준비하는 것이 좋습니다.下, \

[PROMPT]: ### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):
[OUTPUT]: '1954년이다. 항목:\n\n1954년 대선, 리처드 닉슨이 54대 부통령직을 수행한 년도는 1951년이다. 리처드 닉슨은 1950년대 후반부터 1960년대 초반까지 대선에서 후보자로 출마하였으며, 부통령 후보로 출마하는 경우 자신의 정치적 목적을 이루려 노력하였습니다. 항목:\n\n1954년 대선, 리처드 닉슨은 41대 부통령직을 수행하였다. 항목:\n\n1955년 대선, 리처드 닉슨은 40대 부통령직을 수행하였습니다. 항목:\n\n1955년 대선, 리처드 닉슨은 40대 부통령을 역임하였습니다. 항목:\n\

[PROMPT]: ### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):
[OUTPUT]: '시카고 오헤어 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다.: Young Huawei First - 미국 일리노이 주 시카고에 위치해 있습니다.: Young Huawei Young First - 미국 일리노이 주 시카고에 위치해 있습니다.: 선거인단 모

# 최종 출력

SFT와 DPO에서 eos 토큰이 다르기 때문에, 모델이 의도와 다르게 너무 일찍 멈추거나, 끝나도 계속 생성합니다.

모델이 생성한 output_ids는 \[프롬프트 토큰\] + \[응답 토큰들\]로 구성되었기 때문에,

eos를 신뢰하지 않고 프롬프트 이후 토큰만 자르는 방식을 통해

순수한 응답만 남겨지게 됩니다.

In [39]:
PROMPT_DICT = {
    "prompt_input": "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
}

list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    '3+5=?',
    '오늘 비가 내리는데 우산을 챙길까?',
    '인공지능의 미래를 설명해줘'
]

def generation(prompt_text, model, tokenizer, max_new_tokens=128):
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.cuda()
    prompt_len = input_ids.shape[-1]

    # generate 결과 (prompt + response 전체)
    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # 출력에서 prompt 이후만 추출 (응답만)
    response_ids = output_ids[0][prompt_len:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()

    print("Prompt:", prompt_text)
    print("Response:", response_text)
    print("=" * 80)
    
print(tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
    
# 실행
for prompt in list_prompt:
    formatted_prompt = PROMPT_DICT["prompt_input"].format_map({"prompt": prompt})
    generation(formatted_prompt, actor, tokenizer)


1
1
Prompt: ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):
Response: '저는 인공지능 챗봇이며, 실제로 고기를 판매하지 않기 때문에 불고기용 고기의 한우는 판매하지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다.
Prompt: ### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):
Response: '1953년입니다.
Prompt: ### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(응답):
Response: '시카고 오헤어 국제공항은 미국 일리노이 주 시카고에 위치해 있습니다.
Prompt: ### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):
Response: '미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과 실외 활동 자제를 권장합니다. 또한, 미세먼지 농도는 어제와 비교해서 개선되었지만 아직도 나쁜 수준이며, 마스크 착용과 실외 활동 자제를 권장합니다.
Prompt: ### Instruction(명령어):
3+5=?

### Response(응답):
Response: '제가 AI 챗봇이기 때문에, 질문이 이해되지 않습니다. 어떤 내용에 대해 질문하시는지 자세히 설명해주시면 더 정확한 답변을 드릴 수 있을 것 같습니다. 감사합니다.
Prompt: ### Instruction(명령어):
오늘 비가 내리는데 우산을 챙길까?

### Response(응답):
Response: '저는 AI 어시스턴트이기 때문에 비가 내리는지 여부를 판단할 수 없습니다. 하지만 비가 내리는데 우산을 챙겨야 하는 경우도 있겠죠. 비가 내리면 우산이나 방수 장비를 준비하는 것이 좋습니다.
Prompt: ### Instruction(명령어):
인공지능의 미래를 설명해줘



# 회고

코드가 여러 문서에 분산되어있고, 내용 자체도 어려워서 공부하기 어려웠던 노드였습니다.

하지만, 새로운 방식을 적용하면서 여러 문서들을 확인하고, 

왜 목표하던 결과가 나오지 않는지를 분석하면서 기존의 데이터를 확인하고 가공하는 과정이

마지막 적절한 결과가 나오는 것을 통해 즐겁게 마무리할 수 있었습니다.